In [177]:
import pandas as pd
import numpy as np
import cvxpy as cvx
import random
import time
from __future__ import print_function
from builtins import input


from sporco.admm import bpdn
### using the ADMM algorithm 
### we can also use the PGM algorithm 
from sporco import util
from sporco import plot
plot.config_notebook_plotting()

from scipy.linalg import hadamard

import matplotlib.pyplot as plt

from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import KFold

import time 
from scipy import stats
from matplotlib import pyplot
iHiV = pd.read_pickle("~/bge-analysis-simv3/iHiV.pkl")
FT = pd.read_pickle("~/bge-analysis-simv3/Ft.pkl")
B = pd.read_pickle("~/bge-analysis-simv3/B.pkl")
A =  pd.read_pickle("~/bge-analysis-simv3/A.pkl")

import warnings
warnings.filterwarnings("ignore")

In [178]:

tableg=A.T
species_onesall = [] 
for species in range(16):
    species_onesall.append(tableg[tableg[species]==1.0][species].index.to_list())
species_zerosall = []
for species in range(16):
    species_zerosall.append(tableg[tableg[species]==0.0][species].index.to_list())

##### compile the well numbers for each species for its presence 

In [179]:
n = 2**16
random_state = None
opt = bpdn.BPDN.Options({'Verbose': False, 'MaxMainIter': 500,
                         'RelStopTol': 1e-8, 'AutoRho': {'RsdlTarget': 1.0}})

sst_redv1 = pd.read_pickle("~/compressed_sensingv1/realdatasets/OV-16sp-FUNCTIONS/16-list1-allmetabols.pkl")


metabolite_list = ["butyrate"]

In [180]:
ssb = sst_redv1.sort_values(by="butyrate",ascending=False)

In [181]:
ssb

,2,3,5,6,7,9,10,11,15,16,...,18,19,20,21,22,well index,butyrate,acetate,lactate,succinate
173,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.002212,0.0,...,0.000000,0.207881,0.027280,0.000000,0.326747,16780312.0,52.533050,15.978650,2.324100,5.400550
47,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.230445,0.028730,0.000000,0.266936,16780305.0,50.387250,17.803250,3.565250,10.234450
40,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.581517,0.000000,0.000000,0.000000,16778240.0,48.236033,23.496767,3.190200,7.167800
33,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.447153,16779264.0,48.019075,10.242950,19.816612,4.885637
167,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.001616,0.0,...,0.000000,0.000000,0.016142,0.000137,0.436299,16779289.0,47.722200,12.313850,21.412150,3.067500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32768,0.998179,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,16384.0,0.000000,19.815943,22.308886,28.168200
32896,0.926049,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.072593,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,16392.0,0.000000,22.448700,24.365300,32.271700
8192,0.000000,0.0,0.996598,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,65536.0,0.000000,20.064014,33.528329,26.114929
16,0.000000,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.997101,0.000000,0.000000,0.000000,0.000000,1048576.0,0.000000,29.332033,34.575833,0.000000


In [183]:
paa = pd.read_pickle("~/compressed_sensingv1/realdatasets/misc/ophelli-ryan/eff-16-list1-red-pa.pkl")
paa[paa["well number"]==512]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,well number
128,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,524288.0,512


In [184]:
ovd_sps = ["B. vulgatus","B. fragilis", "B. thetai\notaonmicron", "B. caccae","B. cellulos\ninyticus","D. piger","B. longum","B. adoles\ncentis","F. praus\nnitzii",
"C. hiranonis",
"A. caccae",
"B. hydrogeno\ntrophica",
"C.asparagi\nforme",
"E. rectale",
"R. intesti\nnalis",
"C. comes"]

In [185]:
ovd_sps[5],ovd_sps[10]

('D. piger', 'A. caccae')

In [186]:
whper = [] 
linper = [] 
for cv_splits in [10]:
    for metabol in metabolite_list:
        count = 0 
        datab = sst_redv1[str(metabol)]
        databdf = pd.DataFrame(datab)
        datap = list(databdf.index)
        runs = 1
        xs = list(datap)
        random.shuffle(xs)
        data_present = np.array(xs)
        kf = KFold(n_splits=cv_splits)
        kf.get_n_splits(data_present)
        KFold(n_splits=cv_splits, random_state=random_state, shuffle=True)
        for lmda in [7.000]:
            for train_index, test_index in kf.split(data_present):
                X_train, X_test = data_present[train_index], data_present[test_index]
                rinew = list(X_train)
                rileft = list(X_test)
                D = iHiV[rinew,:]
                y2 = databdf.T[rinew].T.values
                yact = databdf.T[rileft].T.values
                s3 = y2 
                b = bpdn.BPDN(D, s3, lmda, opt)
                x = b.solve()
                yrecon = np.dot(iHiV,x.ravel())
                whper.append([runs,lmda,str(metabol),yrecon[rileft],yact,rileft,rinew,cv_splits,len(rileft),len(rinew),x])
                count = count + 1
                print(count)

1


KeyboardInterrupt: 

In [ ]:
whdf = pd.DataFrame(whper)

In [98]:
whdf.to_pickle("optimal_10cv_"+str(metabol)+"_run.pkl")

In [99]:
### looks at the inferred betas

In [100]:
betasall = [] 
for ff in range(10):
    betasall.append(whdf[10][ff].ravel())

In [121]:
#pd.to_pickle(betasall,"inferred_betas_10cvlactate_optimal.pkl")

In [187]:
import pandas as pd
bdf = pd.read_pickle("inferred_betas_10cvbutyrate_optimal.pkl")

In [188]:
bdf2 = pd.DataFrame(bdf[1])
bdf2

,0
0,6.520177
1,0.000000
2,0.000000
3,-0.324624
4,1.006081
...,...
65531,0.000000
65532,0.000000
65533,0.000000
65534,0.000000


In [189]:
bs = np.dot(FT,np.dot(iHiV,bdf2[0]))

In [190]:
whcoeff = pd.DataFrame(bs)

In [191]:
varwh = ((whcoeff[1:])**2)[0].sum()
varwh

137.5468214809617

In [192]:
lrough = [] 
for i in range(16):
    lrough.append(((whcoeff.T[species_onesall[i]].T**2)[0].sum())/varwh)

In [193]:
lrdf = pd.DataFrame(lrough)
lrdf[0].sum()

1.2882139814341702

In [194]:
lrdf

,0
0,0.043321
1,0.030417
2,0.018701
3,0.034907
4,0.027931
5,0.351162
6,0.033281
7,0.015961
8,0.012245
9,0.012881


In [ ]:

betaplot = pd.read_pickle("~/compressed_sensingv1/abun_simv3/rel_abun_betas/rep_"+str(replica)+"RELabun_"+str(species)+"_bge.pkl")
#betaplot = pd.read_pickle("~/compressed_sensingv1/abun_simv3/rel_abun_betas/rep_"+str(replica)+"RELabun_"+str(species)+"_bge.pkl")
df =  pd.read_pickle("~/compressed_sensingv1/abun_simv3/rel_abun_betas/rep_"+str(replica)+"RELabun_"+str(species)+"_bge.pkl")
Bdf = pd.DataFrame(B)
bnew = Bdf.T[species_ones[species]].T
bnew.index = np.arange(0,2**15,1)
dnew = df.T[species_ones[species]].T
dnew.index = np.arange(0,2**15,1)
dnew["order"] = bnew
dnew['weighted'] = dnew[0]*2**(7.5-dnew['order'])
dnewT = dnew.T
hist2 = dnew.sort_values(by=['order'])
histn = hist2.copy()
histns = abs(histn).sort_values(by=['weighted'])
histx = histns.sort_values(by='weighted',ascending=False)